In [1]:
# Change the directory path
%cd ../featureSelection/

C:\Users\mikec\Codes\DAADRISE_AbusiveLangProject\featureSelection


In [2]:
import pandas as pd
import h2o
h2o.init()
from h2o.automl import H2OAutoML
from sklearn import metrics

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.211-b12, mixed mode)
  Starting server from C:\Users\mikec\AppData\Local\Programs\Python\Python37\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\mikec\AppData\Local\Temp\tmpd0watfmm
  JVM stdout: C:\Users\mikec\AppData\Local\Temp\tmpd0watfmm\h2o_mikec_started_from_python.out
  JVM stderr: C:\Users\mikec\AppData\Local\Temp\tmpd0watfmm\h2o_mikec_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Europe/Berlin
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.5
H2O cluster version age:,"28 days, 13 hours and 4 minutes"
H2O cluster name:,H2O_from_python_mikec_4lnfa3
H2O cluster total nodes:,1
H2O cluster free memory:,1.752 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [3]:
X_train = h2o.H2OFrame(pd.read_csv('C:\\Users\\mikec\\Documents\\UnivariateFeatures.csv'))
y_train = h2o.H2OFrame(pd.read_csv('C:\\Users\\mikec\\Documents\\TrainLabels.csv'))
X_test = h2o.H2OFrame(pd.read_csv('C:\\Users\\mikec\\Documents\\UnivariateFeaturesTest.csv'))
y_test = h2o.H2OFrame(pd.read_csv('C:\\Users\\mikec\\Documents\\TestLabels.csv')) 


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
# preparing the train and test data sets
# now convert tweet vecs and labels to a pandas dataframe and back to h2o dataframe
# data = job_titles[valid_job_titles,:].cbind(job_title_vecs[valid_job_titles,:])
train = X_train.cbind(y_train)
test = X_test.cbind(y_test)


In [5]:
# more on data prep
x = train.columns         # x: A list/vector of predictor column names or indexes. 
                          # This argument only needs to be specified if the user wants to exclude columns from the 
                          # set of predictors. If all columns (other than the response) should be used in prediction, 
                          # then this does not need to be set.

y = "labels"              # This argument is the name (or index) of the response column
x.remove(y)

# need to set train and test
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [6]:
# now the AUTO-ML piece comes in
aml = H2OAutoML(max_models=50, max_runtime_secs=7200) #max_models=10 or 20?, max_runtime_secs=3600
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [7]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_per_class_error,logloss,rmse,mse
DeepLearning_grid_1_AutoML_20190717_125621_model_2,0.431079,1.09107,0.484453,0.234694
DeepLearning_grid_1_AutoML_20190717_125621_model_15,0.474999,0.602242,0.404667,0.163755
DeepLearning_grid_1_AutoML_20190717_125621_model_7,0.485857,0.459047,0.386341,0.149259
DeepLearning_grid_1_AutoML_20190717_125621_model_11,0.487591,0.505001,0.394459,0.155598
DeepLearning_1_AutoML_20190717_125621,0.488949,0.50868,0.405623,0.16453
DeepLearning_grid_1_AutoML_20190717_125621_model_1,0.492261,0.669651,0.393758,0.155046
GBM_grid_1_AutoML_20190717_125621_model_8,0.494283,1.1085,0.422584,0.178578
DeepLearning_grid_1_AutoML_20190717_125621_model_4,0.49503,0.643155,0.388122,0.150639
GBM_grid_1_AutoML_20190717_125621_model_17,0.49529,0.458567,0.379415,0.143956
StackedEnsemble_BestOfFamily_AutoML_20190717_125621,0.495475,0.469143,0.382715,0.146471


In [8]:
# The leader model is stored here
aml.leader

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_grid_1_AutoML_20190717_125621_model_2


ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.17377026831159198
RMSE: 0.4168576115553031
LogLoss: 1.4312961490123934
Mean Per-Class Error: 0.5266522892383726
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



0,1,2,Error,Rate
104.0,442.0,29.0,0.8191304,471 / 575
144.0,7350.0,294.0,0.0562404,"438 / 7,788"
9.0,1174.0,496.0,0.7045861,"1,183 / 1,679"
257.0,8966.0,819.0,0.2083250,"2,092 / 10,042"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.791675
2,0.972615
3,1.0



ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.23469449496396133
RMSE: 0.48445277887938815
LogLoss: 1.0910708130104996
Mean Per-Class Error: 0.4310788398574703
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



...

Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_classification_error
,2019-07-17 13:17:06,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan
,2019-07-17 13:17:18,9 min 25.221 sec,7371 obs/sec,4.4472435,1,88091.0,0.4102250,0.8373727,0.2075912,0.1952798
,2019-07-17 13:17:27,9 min 33.656 sec,8680 obs/sec,8.8844911,2,175984.0,0.4127430,0.9308020,0.1978333,0.1919936
,2019-07-17 13:17:35,9 min 41.966 sec,9278 obs/sec,13.3345113,3,264130.0,0.4132787,0.9134517,0.1957500,0.2056363
,2019-07-17 13:17:43,9 min 50.353 sec,9585 obs/sec,17.7818558,4,352223.0,0.4025072,0.8315833,0.2371266,0.1975702
,2019-07-17 13:17:52,9 min 58.773 sec,9770 obs/sec,22.2216781,5,440167.0,0.4394086,1.3855337,0.0908360,0.2018522
,2019-07-17 13:18:00,10 min 7.138 sec,9911 obs/sec,26.6646305,6,528173.0,0.4366449,1.2978832,0.1022366,0.2009560
,2019-07-17 13:18:08,10 min 15.427 sec,10023 obs/sec,31.0979907,7,615989.0,0.4206748,1.2953063,0.1667062,0.1904999
,2019-07-17 13:18:17,10 min 23.735 sec,10107 obs/sec,35.5378635,8,703934.0,0.4094933,0.9287643,0.2104155,0.1953794
,2019-07-17 13:18:25,10 min 32.108 sec,10165 obs/sec,39.9769285,9,791863.0,0.4498225,0.9604203,0.0472313,0.2766381


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
h _tc,1.0,1.0,0.0135505
sentiment,0.9759458,0.9759458,0.0132245
ra_nc,0.9508978,0.9508978,0.0128851
bird_tw,0.9192936,0.9192936,0.0124569
it_tc,0.9004003,0.9004003,0.0122009
---,---,---,---
rash_tc,0.5514030,0.5514030,0.0074718
tra_nc,0.5472590,0.5472590,0.0074156
ash _tc,0.5436653,0.5436653,0.0073669
tra_tc,0.5125073,0.5125073,0.0069447



See the whole table with table.as_data_frame()


In [9]:
preds = aml.predict(test)
print(preds)

deeplearning prediction progress: |███████████████████████████████████████| 100%


predict,p0,p1,p2
0,1,1.79288e-76,1.3688e-35
1,3.21522e-62,1,3.26164e-16
0,1,1.74052e-65,5.80662e-37
2,2.0136e-44,2.55895e-05,0.999974
0,1,3.60704e-70,5.20002e-37
0,1,5.68314e-61,9.50122e-43
1,1.07561e-151,1,2.37255e-13
2,1.68638e-62,8.34357e-09,1
0,1,1.67438e-10,2.58671e-10
1,1.47424e-44,0.998873,0.00112678


In [10]:
var = preds["predict"].cbind(test[y])
print(var)

predict,labels
0,2
1,1
0,2
2,1
0,1
0,2
1,1
2,1
0,2
1,1


In [11]:
# convert to pandas dataframe
y_test = h2o.as_list(test[y], use_pandas=True)
y_pred = h2o.as_list(preds["predict"])
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.accuracy_score(y_test, y_pred))

[[ 136  101   48]
 [ 807 1972 1057]
 [ 446  233  152]]
0.45638126009693053
